This tutorial walks you through how to use the gizmo_analysis package to read particle data from Gizmo simulation snapshot files and from additional post-processing files. It covers the basics of the particle data formats and units. See gizmo_tutorial_analysis.ipynb for a tutorial on analyzing and plotting this particle data.

@author:
    Andrew Wetzel <arwetzel@gmail.com>
    Andrew Emerick <aemerick11@gmail.com>
    Isaiah Santistevan <ibsantistevan@ucdavis.edu>


## Download the _gizmo_analysis_ and _utilities_ python packages

First, download or clone the _gizmo_analysis_ and _utilities_ python packages from:
-  _gizmo_analysis_ : https://bitbucket.org/awetzel/gizmo_analysis
-  _utilities_ : https://bitbucket.org/awetzel/utilities

You can clone them via Git by typing "git clone __[repository link above]__" in your terminal, or you can download the repository via bitbucket by clicking on the ellipsis (the "..." button) and clicking "Download repository".


## Set your paths

Ensure that the "*gizmo_analysis*" and "*utilities*" package directories are in your Python path. This will make sure that python knows where these packages are. The simplest way to ensure this is to launch this notebook (or any python code) from within the directly where the *gizmo_analysis*/" and "*utilities*/ directories are. The better approach is to put the *gizmo_analysis*/" and "*utilities*/ directories in a directory that your python path includes. You may need to look up which file you need to define the paths in. One common place is in a file called the "**.bashrc**" file, which is typically stored in your home directory. (It may not be visible with a simple "ls" command; try using "ls -a" instead.)

An example of what is in my "**.bashrc**" file is:

export HOME="/Users/awetzel"

export PYTHONPATH=$HOME/anaconda/bin:$HOME/analysis

The first line points my home directory, and the second line points to the directory where I have python installed, as well as the directory where I have the simulation data, "*gizmo_analysis*", and "*utilities*" package directories saved. *You will need to make sure that your paths point to your own home directory and the directory where you save the data and analysis packages.*


## Simulation directory

Move within a simulation directory, or set the *simulation_directory* parameter below to point to one. This simulation directory should contain:
- __output/__ - subdirectory that contains Gizmo snapshot files: __snapshot_NNN.hdf5__ or __snapdir_NNN/snapshot_NNN.B.hdf5__, where NNN is the snapshot index, and B is the snapshot file block index
- __snapshot_times.txt__ - text file that lists the indices, scale-factors, redshifts, and times of all snapshots stored from the simulation
- (optionally but ideally) __initial_condition/__ - subdirectory that contains a MUSIC configuration file named __*.conf__ that stores all 6 cosmological parameters. If the simulation directory does not contain, this, GizmoAnalysis will assume the same cosmological parameters as in the AGORA simulation for whichever parameters it cannot read from the Gizmo snapshot header.
- (optionally but ideally) __track/__ - subdirectory that constains a file __host_coordinates.hdf5__ that contains the position, velocity, and orientation of the host galaxy, along with the position and velocity of all star particles when they formed.


In [ ]:
import gizmo_analysis as gizmo  # rename these packages for brevity
import utilities as ut  # rename these packages for brevity

import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
# You can access the files as named or use the aliases in __init__.py to keep it shorter 
# For example, these are the same:

gizmo.gizmo_io
gizmo.io

# Read the particle data

You can copy this jupyter notebook tutorial into a simulation directory (for example, m12i_r7100/) and run from there, or you can set simulation_directory below to point to any simulation directory and then run this notebook from anywhere.

In [ ]:
# Use this is you are running from within a simulation directory
#simulation_directory = '.'

# Use this to point to a specific simulation directory, if you run this notebook from somwhere else
simulation_directory = '/Users/awetzel/work/research/simulation/gizmo/simulations/m12/m12i/m12i_r7100'

In [ ]:
# Read star and dark-matter particles at z = 0, store as a python dictionary

part = gizmo.io.Read.read_snapshots(['star', 'dark'], 'redshift', 0, simulation_directory)

In [ ]:
# Alternately, read all particle species at z = 0

part = gizmo.io.Read.read_snapshots('all', 'redshift', 0, simulation_directory)

part is a dictionary with each particle species name as key:
* 'star' = stars
* 'gas' = gas
* 'dark' = dark matter
* 'dark2', 'dark3', 'dark4', etc = low-resolution dark matter (you rarely will be interested in it)


In [ ]:
part.keys()

In [ ]:
# Each species is its own dictionary that contains arrays of properties

part['star'].keys()

In [ ]:
# List the properties of dark-matter particles

part['dark'].keys()

In [ ]:
# List the properties of gas cells

part['gas'].keys()

# Properties of particles

## Units

Unless otherwise noted, all quantities are in (combinations of):
* mass [M_sun]
* distance, radius [kpc physical]
* velocity [km/s]
* time [Gyr]


## Stored properties

All of the particle species have the following properties:
-  'position' : 3D position along x, y, z grid [kpc comoving]
-  'velocity' : 3D velocity along x, y, z grid [km/s]
-  'mass'     : [Msun]

Star particles also have:
-  'form.scalefactor' : the scale-factor when the star particle formed (goes from 0 [Big Bang] to 1 [today])

Gas cells also have:
-  'temperature' : (K)
-  'density' : [Msun / kpc^3]
-  'sfr' : star-formation rate [Msun / yr]

In [ ]:
# 3-D position of each star particle [kpc comoving]
# stored via a particle_number x dimension_number array

part['star']['position']

In [ ]:
# 3-D velocity of each star particle (particle_number x dimension_number array) [km/s]

part['star']['velocity']

In [ ]:
# Mass of each star particle [M_sun]
# All particles of a given type (star, gas, or dark) all have (about) the same mass, by design.
# For stars and gas, they are not exact, because stellar mass loss transfers mass from star 
# particles to gas cells.

part['star']['mass']

In [ ]:
# Formation scale-factor of each star particle

part['star']['form.scalefactor']

## Derived properties

You can use __.prop('*property_name*')__ to derive other (more useful) quantities.

Some properties that you may want include:
-  'host.distance' : 3D distance of particles from the center of the galaxy in x,y,z grid [kpc physical]
-  'host.distance.total' : Total (1D) distance from the center of the galaxy [kpc physical]
-  'host.velocity' : 3D velocity of particles with respect to the center of the galaxy in x,y,z grid [km/s]
-  'host.velocity.total' : Total (scalar) velocity [km/s]

Star particles also have:
-  'form.time' : Age of the Universe when the star particle formed [Gyr]
-  'age' : Age of the star particle at the current snapshot [Gyr]

Gas particles also have:
-  'number.density' : [hydrogen atoms / cm^3]

See the documentation in the gizmo_io.py file for all options of derived quantities.

Here are a few examples of how to look at properties of star particles.

In [ ]:
# Get the formation time of the star particles [Gyr]

part['star'].prop('form.time')

In [ ]:
# Similarly, get the age of each star particle (the lookback time to when it formed) [Gyr]

part['star'].prop('age')

In [ ]:
# Get the mass of each star particle when it formed [Msun], 
# using the stellar evolution tracks in Gizmo
# this initially sets up an internally stored spline to compute this

part['star'].prop('form.mass')

## Elemental abundances (metallicities)

Elemental abundances are stored in the particle catalog as linear mass fractions one value for each element, in a particle_number x element_number array. The first value is the mass fraction of all metals (everything not H, He) then He, C, N, O, etc.

In [ ]:
part['star']['massfraction']

In [ ]:
# Get individual elements by their array index

# Total metal mass fraction (everything not H, He) is index 0
print(part['star']['massfraction'][:, 0])

# Iron is index 10
print(part['star']['massfraction'][:, 10])

More conveniently, use .prop() to compute derived quantities, including calling element by its name or symbol. See gizmo.io.ParticleDictionaryClass for all options for derived quantities.

In [ ]:
print(part['star'].prop('massfraction.metals'))
print(part['star'].prop('massfraction.carbon'))
print(part['star'].prop('massfraction.iron'))
print(part['star'].prop('massfraction.fe'))  # you can use name or symbol

Also use .prop() to compute 'metallicity' [Z / H].

For example, iron abundance:

$[Fe / H] = \log10[(mass_{Fe} / mass_{H})_{particle} / (mass_{Fe} / mass_{H})_{sun}]$.

GizmoAnalysis assumes solar abundances from Asplund et al 2009.

In [ ]:
print(part['star'].prop('metallicity.metals'))
print(part['star'].prop('metallicity.fe'))

In [ ]:
# You also can use .prop() to compute simple arithmetic combinations, such as [Mg / Fe]

part['star'].prop('metallicity.mg - metallicity.fe')

In [ ]:
# Refer to utilities.constant for assumed solar values (Asplund et al 2009) and other constants.

ut.constant.sun_abundance
ut.constant.sun_massfraction

# Metadata about simulation

The dictionary that stores the particle catalog is actually a dictionary class. It also stores meta-data about the simulation and ancillary data/functions via appended dictionaries, classes, and arrays.

In [ ]:
# Dictionary of useful information about the simulation

part.info

In [ ]:
# Dictionary of information about the snapshot that you read:
# its index (number), scale-factor, redshift, time, lookback-time

part.snapshot

In [ ]:
# Dictionary class with information about *all* snapshots (typically ~600) saved for this simulation

print(part.Snapshot.keys())
print(part.Snapshot['redshift'][:10])

In [ ]:
# Dictionary class of cosmological parameters, with internal functions for cosmological conversions
# see utilities.cosmology for more

part.Cosmology

# Coordinates of host galaxy/halo and principal axes of stellar disk

If you enable gizmo.io.Read.read_snapshots(assign_hosts=True), which is True by default, then during read-in, this package assigns the position and velocity of the host galaxy/halo, using stars for a baryonic simulations and dark matter for a dark-matter-only simulation. GizmoAnalysis stores these coordinates in arrays appended to the particle dictionary.

Most simulations have a single host galaxy/halo, but some (like ELVIS simulations of Local Group-like pairs) contain 2 (or more). You can control the number of hosts via: read_snapshots(host_number=2) By deafult, host_number=1.

Once the code assigns the coordinates of each host, it also can compute the 'orientation' of the disk, by computing the rotation tensor and axis ratios of the principal axes of each host's stellar disk, defined via the moment of inertia tensor of young star particles, and oriented so the median v_phi of young star particles is positive. Enable this via: read_snapshots(assign_hosts_rotation=True). By default, assign_hosts_rotation=False.

If someone has run particle tracking on the simulation, including generating a file track/host_coordinates.hdf5, this stores the posiion, velocity, rotation tensor, and axis ratio of each host galaxy at each snapshot, so you do not have to re-compute during every read. If you set assign_hosts=True or assign_hosts_rotation=True in read_snapshots(), then this package first will look for this file, and assign the host information stored in it, but if it does not find the file, this package will assign these host properties on the fly during read.

In [ ]:
# Compute (or read) the rotation tensor and axis ratios of the principal axes 
# (defined via moment of inertia tensor) of stars in each host during read as follows

part = gizmo.io.Read.read_snapshots(['star', 'dark'], 'redshift', 0, simulation_directory, assign_hosts_rotation=True)

In [ ]:
# position [kpc comoving] and velocity [km/s] of the center of each host galaxy
# The package can store multiple hosts, though usually just one.

print(part.host['position'])
print(part.host['velocity'])

In [ ]:
# The rotation tensor, used to rotate into the principal axes for each host galaxy

print(part.host['rotation'])

In [ ]:
# The axis ratios (min/maj, min/med, med/maj) for each host galaxy

print(part.host['axis.ratios'])

Now you can compute different types of distances of star particles from the center of the host galaxy using __.prop('*property_name*')__ like in the examples mentioned above.

To get the 3-D distance from the host center along the simulation's default x,y,z cartesian axes, you can use:
-  .prop('host.distance')

To get the 1-D distance (total scalar distance) from the host center, you can use:
-  .prop('host.distance.total')

In [ ]:
# Get the 3-D distance from the host center along simulation's default x,y,z cartesian axes [kpc physical]

part['star'].prop('host.distance')

In [ ]:
# Add '.total' to compute total (scalar) distance [kpc physical]

part['star'].prop('host.distance.total')

To get the 3-D distance aligned with the principal axes of the host's stellar disk, you can use:
- .prop('host.distance.principal')

To get the 3-D distance aligned with the principal axes in cylindrical coordinates, you can use:
- .prop('host.distance.principal.cylindrical')

In [ ]:
# Add '.principal' to compute 3-D distance aligned with the principal 
# (major, intermediate, minor) axes of each host's stellar disk [kpc physical]

part['star'].prop('host.distance.principal')

In [ ]:
# Add '.cylindrical' or '.cyl' to compute 3-D distance aligned with the principal axes in cylindrical coordinates
# First value is along the major axes (R, positive definite) [kpc physical]
# Second value is angle (phi, 0 to 2 * pi) [radian]
# Third value is vertical height wrt the disk (Z, signed) [kpc physical]

part['star'].prop('host.distance.principal.cylindrical')

In [ ]:
# Alternately, add 'spherical' to compute 3-D distance aligned with the principal axes in spherical coordinates
# First value is the radial distance from the galaxy center (r, positive definite)
# Second value is theta (theta, 0 to pi)
# Third value is phi (phi, 0 to 2 * pi)

part['star'].prop('host.distance.principal.spherical')

You can use similar commands for the velocities

In [ ]:
# Compute 3-D velocity from each host galaxy center along simulation's default x,y,z cartesian axes [km/s]

part['star'].prop('host.velocity')

In [ ]:
# Compute total (scalar) velocity [km/s]

part['star'].prop('host.velocity.total')

In [ ]:
# Compute 3-D velocity along the principal (major, intermediate, minor) axes [km/s]

part['star'].prop('host.velocity.principal')

In [ ]:
# Compute 3-D velocity in cylindrical coordinates
# First value is along the major axes
# Secod value is azimuthal velocity in the plane of the disk 
# (the disk is oriented) so the mean v_phi should be positive
# Third value is vertical velocity wrt the disk (signed)

part['star'].prop('host.velocity.principal.cylindrical')

In [ ]:
# If you want to store multiple hosts (like the ELVIS Local Group-like paired host simulations), 
# set host_number=2 during read-in.
# In fact, you can do this for any simulation, with an arbitrary number of hosts
# the code simply finds the second (and third, etc) most massive galaxy in the zoom-in region.

part = gizmo.io.Read.read_snapshots(['star', 'dark'], 'redshift', 0, simulation_directory, host_number=2, assign_hosts='mass', assign_hosts_rotation=True)

In [ ]:
# Everything above still works, just use 
# 'host' (or 'host1'), 'host2', 'host3', etc to identify which host you want coordinates relative to

print(part['star'].prop('host.distance'))  # same as part['star'].prop('host1.distance')
print(part['star'].prop('host2.distance'))

In [ ]:
# the particle dictionary stores coordinates and rotation tensors for each host

print(part.host['position'])
print(part.host['velocity'])
print(part.host['rotation'])
print(part.host['axis.ratios'])

# Particle tracking

For some simulations, we have run a post-processing particle tracking pipeline, which makes it easy to track star particles and/or gas cells across time. These simulations have a directory named __track/__, which contains .hdf5 files.

__gizmo_track.py__ contains the code that generates and reads these particle tracking files.

We use this particle tracking pipeline to generate the file __track/host_coordinates.hdf5__, which stores the posiion, velocity, rotation tensor, and axis ratio of each host galaxy at each snapshot. We compute this by tracking back only the particles that end up in each host galaxy/halo at the final snapshot (z = 0).

The code stores each host's coordinates and rotation tensor (for its principal axes) at each snapshot. You may see Nan values at early snapshots, if the code was not able to identify a host at that time.

In [ ]:
print('positions')
print(part.hostz['position'])

print('velocities')
print(part.hostz['velocity'])

print('rotation tensors')
print(part.hostz['rotation'])

print('axis ratios')
print(part.hostz['axis.ratios'])

## Formation coordinates of star particles

The file __track/host_coordinates.hdf5__ also stores, for every star particles at z = 0, its 'formation coordinates', that is, its 3-D position and 3-D velocity at the snapshot immediately after it formed. Because we typically store snapshots every 20 - 25 Myr, this means that these 'formation' coordiantes are the coordinates of a star particle 0 - 25 Myr after it formed. These coordinates are aligned with the principal (major, intermediate, minor) axes of the stellar disk (as defined via its moment of inertia tensor) computed separately at each snapshot.

In [ ]:
# You can read the formation coordinates of star particles during snapshot read-in
# by setting assign_formation_coordinates=True

part = gizmo.io.Read.read_snapshots(
    ['star'], 'redshift', 0, simulation_directory, assign_hosts_rotation=True, 
    assign_formation_coordinates=True)

In [ ]:
# 3-D distance at formation, aligned with the principal axes of each host galaxy at that time [kpc physical]
# The code computed the principal axes *independently* at each snapshot.
# Distance along dimension 0 is aligned with the major axis
# Distance along dimension 1 is algined with the intermediate axis
# Distance along dimension 2 is aligned with the minor (Z) axis

part['star']['form.host.distance']

# part_at_z0['star']['form.host2.distance']  # if the simulation has a second host galaxy (for example, ELVIS)

In [ ]:
# As before, add 'total' to get the total scalar (absolute) distance wrt each host galaxy at formation [kpc physical]
# this is a derived quantity, so you need to call it via .prop().

part['star'].prop('form.host.distance.total')

In [ ]:
# Add 'cylindrical' to get 3-D distance at formation wrt each host galaxy in cylindrical coordinates [kpc physical]

part['star'].prop('form.host.distance.cylindrical')

In [ ]:
# These values are more reasonable if you restrict to star particles that formed within a host galaxy

# Select particles formed at d = 0 - 8 kpc physical
part_indices = ut.array.get_indices(part['star'].prop('form.host.distance.total'), [0, 8])

part['star'].prop('form.host.distance.cylindrical', part_indices)

In [ ]:
# Same for velocity at formation

print(part['star']['form.host.velocity'])
print(part['star'].prop('form.host.velocity.total'))
print(part['star'].prop('form.host.velocity.cylindrical'))

## Tracking star particles and gas cells across snapshots

Within __track/__, the files named __star\_gas\_pointers\_NNN.hdf5__ store, for each star particle and gas cell at z = 0, a pointer to its array index in the catalog at each previous snapshot, NNN. This makes it easy to get the properties of a given star particle at any previous snapshot.

### Tracking between z = 0 and a previous snapshot

Say you want to find out what star particles and gas cells were doing at z = 2. First, read in the particle catalog at z = 0.

In [ ]:
# Read catalog of star particles and gas cells at z = 0

part_at_z0 = gizmo.io.Read.read_snapshots(['star', 'gas'], 'redshift', 0, simulation_directory)

Next, read particles at z = 2. Set assign_pointers=True to assign pointers to the particle catalog at z = 2.

In [ ]:
# Read in catalog of star particles and gas cells at z = 2 (corresponding to snapshot 172)

part_at_z2 = gizmo.io.Read.read_snapshots(
    ['star', 'gas'], 'redshift', 2, simulation_directory, assign_pointers=True)

In [ ]:
# The packaes stores particle pointers via a dictionary class that it appends to the particle catalog dictionary.
# A negative value for a pointer means that the star formed after this snapshot, so it does not exist at this snapshot.

part_at_z2.Pointer

In [ ]:
# The particle species that we have compiled tracking pointers for

part_at_z2.Pointer['species']

Print the snapshot indices at z = 2 and at the reference redshift, which is z = 0

In [ ]:
# The snapshot index at this redshift (z = 2) and at the reference redshift (z = 0)

print(part_at_z2.Pointer['z.snapshot.index'])
print(part_at_z2.Pointer['z0.snapshot.index'])

See the number of particles at z = 2 and at z = 0

In [ ]:
# The number of particles at this redshift (z = 2) and at the reference redshift (z = 0)

print(part_at_z2.Pointer['z.star.number'])
print(part_at_z2.Pointer['z.gas.number'])

print(part_at_z2.Pointer['z0.star.number'])
print(part_at_z2.Pointer['z0.gas.number'])

The dictionaries below store the actual pointer indices. Because gas cells can become star particles, the code stores the pointers from this combined gas + star list at z = 0 to z.

In [ ]:
print(part_at_z2.Pointer['z0.to.z.index'])

# This also stores the limits of the indices of each species at each snapshot, 
# so you can convert back to the index within the individual star list or the individual gas list 
# (as stored in the particle catalog)
print(part_at_z2.Pointer['z.star.index.limits'])
print(part_at_z2.Pointer['z.gas.index.limits'])

print(part_at_z2.Pointer['z0.star.index.limits'])
print(part_at_z2.Pointer['z0.gas.index.limits'])

Even easier, you can use this function to get pointers for a given species from z = 0 to z = 2.

In [ ]:
pointers = part_at_z2.Pointer.get_pointers(species_name_from='star', species_names_to='star')

### A more specific example

Say you have a list of star particle indices of interest at z = 0. Save those indices to an array and print out their positions at z = 0.

In [ ]:
indices_at_z0 = np.array([2, 4])

# list their positions at z = 0
part_at_z0['star']['position'][indices_at_z0]

Now, you can get their indices in the particle catalog at z = 2 using the following command. Remember that negative indices means that the star particle did not exist at z = 2.

In [ ]:
indices_at_z2 = pointers[indices_at_z0]
print(indices_at_z2)

Now you can easily get any property of interest at z = 2. Print their positions:

In [ ]:
part_at_z2['star']['position'][indices_at_z2]

In [ ]:
# Alternatively, you can track particles going forward in time by setting forward=True

pointers = part_at_z2.Pointer.get_pointers(species_name_from='star', species_names_to='star', forward=True)

indices_at_z2 = np.array([5, 8, 13])
indices_at_z0 = pointers[indices_at_z2]
print(indices_at_z0)
print(part_at_z0['star']['position'][indices_at_z0])

In [ ]:
# Also, you can track star particles back to both progenitor star particles and progenitor gas cells

pointers = part_at_z2.Pointer.get_pointers(species_name_from='star', species_names_to=['star', 'gas'])

# But now, pointers is a *dictionary* that stores both the index and species of each progenitor particle
print(pointers)

# get star particle indices at z = 0 and see what they were at z = 2
star_indices_at_z0 = np.array([0, 5, 8, 13])
print(pointers['species'][star_indices_at_z0], pointers['index'][star_indices_at_z0])

# get those that were gas cells
masks = np.where(pointers['species'][star_indices_at_z0] == 'gas')[0]

gas_indices_at_z2 = pointers['index'][star_indices_at_z0[masks]]

print(gas_indices_at_z2)
print(part_at_z2['gas']['position'][gas_indices_at_z2])

In [ ]:
# Similar for working forward in time, 
# track gas cells at z that can be star particles or gas cells at z = 0

pointers = part_at_z2.Pointer.get_pointers(
    species_name_from='gas', species_names_to=['star', 'gas'], forward=True)

# get gas indices at z = 2 and see what they end up as at z = 0
gas_indices_at_z2 = np.array([0, 5, 8, 13])
print(pointers['species'][gas_indices_at_z2])
print(pointers['index'][gas_indices_at_z2])

# get those that are star particles at z = 0
masks = np.where(pointers['species'][gas_indices_at_z2] == 'star')[0]

star_indices_at_z0 = pointers['index'][gas_indices_at_z2[masks]]

print(star_indices_at_z0)
print(part_at_z0['star']['position'][star_indices_at_z0])

### tracking between two snapshots when both are at z > 0

Particle tracking also can handle tracking particles between any two snapshots.

In [ ]:
# Read catalogs of star particles and gas cells at z = 1 and 2, including their pointers relative to z = 0
# set asign_pointers=True automatically to append pointer class to each particle catalog
part_at_z1 = gizmo.io.Read.read_snapshots(
    ['star', 'gas'], 'redshift', 1, simulation_directory, assign_pointers=True)
part_at_z2 = gizmo.io.Read.read_snapshots(
    ['star', 'gas'], 'redshift', 2, simulation_directory, assign_pointers=True)

In [ ]:
# Now just append intermediate-redshift pointers (to z = 1) to pointers at z = 2

part_at_z2.Pointer.add_intermediate_pointers(part_at_z1.Pointer)

In [ ]:
# Now you can access pointers from z = 1 to z = 2, by setting intermediate_snapshot=True

pointers = part_at_z2.Pointer.get_pointers(
    species_name_from='star', species_names_to=['star', 'gas'], intermediate_snapshot=True)

# get star indices at z = 0 and see what they were at z = 2
star_indices_at_z1 = np.array([0, 5, 8, 13])
print(pointers['species'][star_indices_at_z1])
print(pointers['index'][star_indices_at_z1])

# get those that are star particles at z = 2
masks = np.where(pointers['species'][star_indices_at_z1] == 'star')[0]

star_indices_at_z2 = pointers['index'][star_indices_at_z1[masks]]

print(star_indices_at_z2)
print(part_at_z2['star']['position'][star_indices_at_z2])

In [ ]:
# If you just want to track a single species (star -> star or gas -> gas) between 2 snapshots,
# read_pointers_between_snapshots() makes it easy to get the pointer indices between any 2 snapshots

ParticlePointer = gizmo.track.ParticlePointerClass(simulation_directory=simulation_directory)

# tracking forward in time
pointers_z2_to_z1 = ParticlePointer.read_pointers_between_snapshots(
    snapshot_index_from=172, snapshot_index_to=277, species_name='star')

# for example, see how far star particles have moved
print(part_at_z2['star']['position'] - part_at_z1['star']['position'][pointers_z2_to_z1])


# similar for tracking going backward in time
pointers_z1_to_z2 = ParticlePointer.read_pointers_between_snapshots(
    snapshot_index_from=277, snapshot_index_to=172, species_name='star')

# in this case, need to select stars that existed as stars at z = 2
masks = (pointers_z1_to_z2 >= 0)

# see how far star particles have moved
print(part_at_z1['star']['position'][masks] - part_at_z1['star']['position'][pointers_z1_to_z2[masks]])

In [ ]:
# Do the same, but for gas

ParticlePointer = gizmo.track.ParticlePointerClass(simulation_directory=simulation_directory)

# tracking going forward in time
pointers_z2_to_z1 = ParticlePointer.read_pointers_between_snapshots(
    snapshot_index_from=172, snapshot_index_to=277, species_name='gas')

# ensure that gas cell still is a gas cell at z = 1
masks = (pointers_z2_to_z1 >= 0)

print(part_at_z2['gas']['position'][masks] - part_at_z1['gas']['position'][pointers_z2_to_z1[masks]])

# tracking going backward in time
pointers_z1_to_z2 = ParticlePointer.read_pointers_between_snapshots(
    snapshot_index_from=277, snapshot_index_to=172, species_name='gas')

# for gas, still have to ensure positive pointers, even if tracking going backward in time,
# because a few gas cells that leave the zoom-in region get culled (for numerical stability) by z = 0
# (remember that the pointers always route through z = 0)
masks = (pointers_z1_to_z2 >= 0)

print(part_at_z1['gas']['position'][masks] - part_at_z2['gas']['position'][pointers_z1_to_z2[masks]])

# For more information

See *gizmo_tutorial_analysis.py* for a tutorial on analyzing and plotting this particle data.

See *gizmo_plot.py* (which you can access here via *gizmo.plot*) for more examples of analyzing and plotting particle data.

See *utilities/particle.py* (which you can acces here via *ut.particle*) for mid-level analysis functions that may be useful.

See other modules within *utilities* for low-level functions that may be useful.